In [6]:
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import ftfy
import re
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer



In [7]:
input="/home/ndl/Code/data/depression_tweet_dataset_by_model.csv"
g_input="/home/ndl/Downloads/glove.6B.100d.txt"
w2v_output="/home/ndl/Downloads/gensim_glove_vectors.txt"
store="/home/ndl/Project/result/Glove/Cos_Sim/Glove_CosSim_"

In [8]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')


In [9]:
data = pd.read_csv(input, sep=",")

#data = pd.read_csv("/home/ndl/Documents/data.csv", sep=",")

In [10]:
data.head(10)

,id,tweet,created_at,label
0,1.207186e+18,@PrajwalaNarayan @Sirchahal @zainabsikander th...,2019-12-18 06:28:06+00:00,3
1,1.207186e+18,@anshuanand19 Talk about 19 lakh people who ha...,2019-12-18 06:29:53+00:00,2
2,1.207188e+18,6/ In so many countries now that fear of other...,2019-12-18 06:36:00+00:00,2
3,1.207188e+18,@Citizen91712133 @MsIndy313 @SarahHa59297099 @...,2019-12-18 06:36:48+00:00,2
4,1.207188e+18,We're a 5 trillion economy. We're a 1.3B popul...,2019-12-18 06:38:50+00:00,1
5,1.207189e+18,#JamiaProtests #CAA_NRC @WSJ @NewYorker Story ...,2019-12-18 06:41:18+00:00,2
6,1.207191e+18,A baby of orange juice and anxiety https://t.c...,2019-12-18 06:47:18+00:00,2
7,1.207192e+18,@swapan55 Pretty much everyone is against this...,2019-12-18 06:52:43+00:00,2
8,1.207192e+18,Part II Interview Pro Tips continuing from P...,2019-12-18 06:54:03+00:00,2
9,1.207192e+18,"@MdpMinx22 Whatever you are battling, I ask Go...",2019-12-18 06:54:03+00:00,2


In [11]:
depress = data[data.label < 2]
motivation = data[data.label > 2]

#depress=data[data.label=='D']
#motivation=data[data.label=='M']


depress = depress.reset_index()
motivation = motivation.reset_index()

print(depress.shape)
print(motivation.shape)



(470, 5)
(189, 5)


In [12]:
#list of original depress_tweet

depress.head(10)
depress_original_list = depress["tweet"].tolist()
print(depress_original_list)
#print(len(depress_original_list))


["We're a 5 trillion economy. We're a 1.3B population. We're losing jobs,poor is getting poorer,woman isn't safe,st… https://t.co/cuUxSM6V9S", 'It is not because things are bad that you are sad, it is because you are sad that things are bad. Get out of depres… https://t.co/FW4Kh1Z0G1', 'iStressfree – Reduce StressHelps reduce anxiety, stress of strain.https://t.co/NPiqEmbedP#stress #life #love… https://t.co/b1uhHnEd71', '@PrajwalaNarayan @Sirchahal @zainabsikander I never gave any certificate to anyone, if you want my certificate. So… https://t.co/Qooo6X0ap3', "@AsYouNotWish 1)The fear is in the air.2) suicide bomber3) terror attack4) something's fishy5)we r kafirs6)gaz… https://t.co/w7lqPaFF3M", 'Hum students out of stress suicide karte haiTum genocide karte hoWe are not the same bro 🙂#CAA_NRC', 'My favorite one is putting on the reddest lipstick when you like have anxiety at 2am because we’re THAT bitch 🎈 https://t.co/wA9lmq65sa', 'Lack of knowledge or partial knowledge is the founda

In [13]:
#list of original motivation_tweet


motivation.head(10)
motivation_original_list = motivation["tweet"].tolist()
print(motivation_original_list)
print(len(motivation_original_list))


['@PrajwalaNarayan @Sirchahal @zainabsikander the day modi will apologize the nation for riots of 2002, for demonetiz… https://t.co/qVZDCwZPEJ', "@TheOfficialSBI How Banks force to commit suicide impose HOME LOAN HIGH RATE OF INTEREST if u won't give bribe. How… https://t.co/oRVShlsWrW", '@PrajwalaNarayan the day modi will apologize the nation for riots of 2002, for demonetization blunder, the day he w… https://t.co/mRL8hnrLGP', '@AsYouNotWish Honesty suicide bomber', '@twistedSouffle That us what you want to feel to make you feel better.. however god bless you and shayad kuch akal… https://t.co/4rL8JgBmVr', '@Nomthandazo_D @SciTheComedist In case of suicide is also a choice', 'Excellent. But, build public confidence, especially among Muslims, on the NRC process and provisions, and how CAA-N… https://t.co/krquZb30yk', "#UncutGems is the most stressful, anxiety-inducing movie I've ever seen....and I loved every minute of it.", 'Can you like please,  and i ask this with utmost respect,  

In [14]:
# Expand Contraction
cList = {
    "aint": "am not",
    "ain't": "am not",
    "aren't": "are not",
    "cant": "cannot",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didnot": "did not",
    "didnt": "did not",
    "didn't": "did not",
    "doesn't": "does not",
    "dont": "do not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "hows": "how is",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "its": "it is",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "thats": "that is",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "whats": "what is",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "wont": "will not",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are",
    "you've": "you have"
}


In [15]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
c_re = re.compile('(%s)' % '|'.join(cList.keys()))
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)



[nltk_data] Downloading package stopwords to /home/ndl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ndl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ndl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
def clean_each_tweet(tweet):
    
    # if url links then dont append to avoid news articles
    # also check tweet length, save those > 10 (length of word "depression")
    #if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
    #print(FindURL(tweet))
    if len(tweet) > 10:
        
        #remove URLs
        tweet = ' '.join(
            re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " ", tweet).split())

        # remove hashtag, @mention, emoji and image URLs
        #tweet = ' '.join(
        #   re.sub("(@[A-Za-z0-9_]+)|(\#[A-Za-z0-9_]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", tweet).split())

        #Not removing the hashtags but "RT :", @mention, emoji and image URLs
        tweet = ' '.join(
            re.sub("(RT+)|(@[A-Za-z0-9_]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", tweet).split())

        # fix weirdly encoded texts
        tweet = ftfy.fix_text(tweet)

        # expand contraction
        tweet = expandContractions(tweet)

        # remove punctuation
        tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())

        # stop words
        stop_words = set(stopwords.words('english'))
        word_tokens = nltk.word_tokenize(tweet)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        tweet = ' '.join(filtered_sentence)


        # stemming words
        tweet = PorterStemmer().stem(tweet)

        # lemmatize words
        wn = nltk.WordNetLemmatizer()
        tweet = wn.lemmatize(tweet)

        return tweet


In [17]:
depress_cleaned = depress['tweet'].apply(clean_each_tweet)
motivation_cleaned = motivation['tweet'].apply(clean_each_tweet)



In [18]:
#motivation_cleaned 
ml=motivation_cleaned.tolist()
ml
tokenized_sents2 = [word_tokenize(i) for i in ml]
print((tokenized_sents2))

print("---------------------------------------------------------------------")

#depress_cleaned
dl=depress_cleaned.tolist()
dl
tokenized_sents3 = [word_tokenize(i) for i in dl]
print((tokenized_sents3))



[['day', 'modi', 'apologize', 'nation', 'riots', '2002', 'demonetiz'], ['how', 'banks', 'force', 'commit', 'suicide', 'impose', 'home', 'loan', 'high', 'rate', 'of', 'interest', 'u', 'give', 'bribe', 'how'], ['day', 'modi', 'apologize', 'nation', 'riots', '2002', 'demonetization', 'blunder', 'day', 'w'], ['honesty', 'suicide', 'bomb'], ['that', 'us', 'want', 'feel', 'make', 'feel', 'better', 'however', 'god', 'bless', 'shayad', 'kuch', 'ak'], ['in', 'case', 'suicide', 'also', 'choic'], ['excellent', 'but', 'build', 'public', 'confidence', 'especially', 'among', 'muslims', 'nrc', 'process', 'provisions', 'caa', 'n'], ['uncutgems', 'stressful', 'anxiety', 'inducing', 'movie', 'i', 'ever', 'seen', 'i', 'loved', 'every', 'minut'], ['can', 'like', 'please', 'ask', 'utmost', 'respect', 'like', 'go', 'fuck'], ['discuss', 'many', 'people', 'got', 'points', 'thier', 'anxiety', 'cab', 'living', 'family', 'sinc'], ['out', 'fear', 'allah', 'community', 'people', 'pick', 'guns', 'throw', 'stone', '

In [19]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file=g_input, word2vec_output_file=w2v_output)

(400000, 100)

In [20]:
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format(w2v_output, binary=False,limit=500000)

In [21]:
print(glove_model)
print(len(glove_model.vocab))


400000


In [22]:
vocabulary = set(glove_model.index2word)  

In [23]:
#Tidy sentence

from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess    

def tidy_sentence(tokenized_sents, vocabulary):
    return [word for word in tokenized_sents[0] if word in vocabulary]    


In [24]:
#Tidy sentence(depression_tweet)

a1=[]
for i in range(0,len(depress_original_list)):
    a=tidy_sentence(tokenized_sents3[i:i+1],vocabulary)
    a1.append(a)
print(a1)

[['we', '5', 'trillion', 'economy', 'we', '1', '3b', 'population', 'we', 'losing', 'jobs', 'poor', 'getting', 'poorer', 'woman', 'safe', 'st'], ['it', 'things', 'bad', 'sad', 'sad', 'things', 'bad', 'get'], ['reduce', 'reduce', 'anxiety', 'stress', 'strain', 'stress', 'life', 'lov'], ['i', 'never', 'gave', 'certificate', 'anyone', 'want', 'certificate', 'so'], ['1', 'the', 'fear', 'air', '2', 'suicide', 'terror', 'something', 'r', 'gaz'], ['hum', 'students', 'stress', 'suicide', 'karte', 'genocide', 'karte', 'howe', 'bro', 'caa', 'nrc'], ['my', 'favorite', 'one', 'putting', 'reddest', 'lipstick', 'like', 'anxiety', '2am', 'that', 'bitch'], ['lack', 'knowledge', 'partial', 'knowledge', 'foundation', 'life', 'failures', 'destruction', 'depression'], ['suicide', 'bomber', 'stupid', 'in'], ['historians', 'word', 'germans', 'joined', 'nazi', 'party', 'hated', 'jews', 'hope', 'rest'], ['suicide', 'takes', 'pain', 'away', 'it', 'transfers', 'someone', 'else', 'suicide', 'opt'], ['sir', 'compl

In [25]:
#Tidy sentence(motivation_tweet)


a2=[]
for i in range(0,len(motivation_original_list)):
    a0=tidy_sentence(tokenized_sents2[i:i+1],vocabulary)
    a2.append(a0)
print(a2)

[['day', 'modi', 'apologize', 'nation', 'riots', '2002'], ['how', 'banks', 'force', 'commit', 'suicide', 'impose', 'home', 'loan', 'high', 'rate', 'of', 'interest', 'u', 'give', 'bribe', 'how'], ['day', 'modi', 'apologize', 'nation', 'riots', '2002', 'demonetization', 'blunder', 'day', 'w'], ['honesty', 'suicide', 'bomb'], ['that', 'us', 'want', 'feel', 'make', 'feel', 'better', 'however', 'god', 'bless', 'kuch', 'ak'], ['in', 'case', 'suicide', 'also'], ['excellent', 'but', 'build', 'public', 'confidence', 'especially', 'among', 'muslims', 'nrc', 'process', 'provisions', 'caa', 'n'], ['stressful', 'anxiety', 'inducing', 'movie', 'i', 'ever', 'seen', 'i', 'loved', 'every'], ['can', 'like', 'please', 'ask', 'utmost', 'respect', 'like', 'go', 'fuck'], ['discuss', 'many', 'people', 'got', 'points', 'thier', 'anxiety', 'cab', 'living', 'family', 'sinc'], ['out', 'fear', 'allah', 'community', 'people', 'pick', 'guns', 'throw', 'stone', 'become', 'suicide', 'bombers', 'wow'], ['haan', 'reali

In [26]:
d_list=depress_cleaned.tolist()
d_list[0]
#m_list=motivation_cleaned.tolist()
#m_list[2]

'we 5 trillion economy we 1 3b population we losing jobs poor getting poorer woman safe st'

In [27]:
#glove similarity function

def glove_similarity(a1,a2,max):
    s1=[]
    for i in range(0,len(motivation_original_list)):
            s=glove_model.n_similarity(a1[0],a2[i])
            s1.append(s)
    #print(s1)
    #print(len(s1))
    
    df0 = pd.DataFrame(s1,columns =['similarity'])
    #df0
    df0['motivation_tweet']=motivation_original_list
    df0
    df0.sort_values("similarity", axis = 0, ascending = False, 
                 inplace = True, na_position ='last') 
    
    
    df3=df0.nlargest(max, ['similarity'])
    return(df3)

In [28]:
import itertools

In [32]:
for i in range(0,len(depress_original_list)):
        g=glove_similarity(a1[i:i+1],a2 ,5)
        #print(res)
        l=list(g.index.values) 
        g["index of motivation tweet"]=l
        row=g.count(axis = 0)
        #res['depression_tweet']=list(itertools.repeat(d_list[i], 5))
        g['depression_tweet']=list(itertools.repeat(depress_original_list[i], row[0]))
        print(g)
        #g.to_csv("/home/ndl/Result/Glove/Cos_Sim/Glove_CosSim_"+str(i)+".csv",index=None)
        g.to_csv(store+str(i)+".csv",index=None)
        
        #g.to_csv("/home/ndl/Result2/Glove/Cos_Sim/Glove_CosSim_"+str(i)+".csv",index=None)

     similarity                                   motivation_tweet  \
81     0.900144  2020 gonna b so private, quiet, happy and full...   
131    0.899058  @Huda_Nav This Year Is The Year Of Depression ...   
182    0.880197  @bbclaurak This may encourage people to think ...   
69     0.876745  not a great time for depression to hit me this...   
63     0.874902  our Great Depression is our lives...We've all ...   

     index of motivation tweet  \
81                          81   
131                        131   
182                        182   
69                          69   
63                          63   

                                      depression_tweet  
81   We're a 5 trillion economy. We're a 1.3B popul...  
131  We're a 5 trillion economy. We're a 1.3B popul...  
182  We're a 5 trillion economy. We're a 1.3B popul...  
69   We're a 5 trillion economy. We're a 1.3B popul...  
63   We're a 5 trillion economy. We're a 1.3B popul...  
     similarity                 

In [30]:

n1=[124, 368, 137, 226, 438]

In [31]:
#function for calculating precision


def cal_precision(d1):
    r = d1["Relevant"].tolist()
    r
    c=0
    for i in r:
        if i==1:
            c=c+1
    #c
    precision0=c/len(r)
    #print(precision0)
    return(precision0)


In [26]:
precision_list=[]
for i in n1:

        #d1=pd.read_csv('/home/ndl/Result3/Glove_CosSim_'+str(i)+'.csv')
        d1=pd.read_csv('/home/ndl/Result4/Glove_CosSim_'+str(i)+'.csv')
        precision_list.append(cal_precision(d1))
print(precision_list)


[0.6, 0.6, 0.6, 0.4, 0.6]


In [27]:
#Avg precision for glove
sum=0.000
for i in precision_list:
    sum=sum+i
sum
print(sum/5)

0.5599999999999999


In [30]:
#Fnction for calculating MRR_list

def mrr(d2,c):
    #c=1
    for i in d2:
        if i==1:
            #print(c)
            return (1/c)
            break
        c=c+1
    

In [31]:
#MRR
mrr_list=[]
for i in n1:

        d1=pd.read_csv('/home/ndl/Result4/Glove_CosSim_'+str(i)+'.csv')
        d2=d1['Relevant'].tolist()
        print(d2)
        a0=mrr(d2,1)
        print(a0)
        mrr_list.append(a0)
        
print(mrr_list)
sum=0
for i in mrr_list:
    sum=sum+i
print(sum/5)


[1, 1, 0, 0, 1]
1.0
[0, 1, 1, 1, 0]
0.5
[1, 1, 0, 0, 1]
1.0
[1, 0, 0, 0, 1]
1.0
[0, 0, 1, 1, 1]
0.3333333333333333
[1.0, 0.5, 1.0, 1.0, 0.3333333333333333]
0.7666666666666667
